<a href="https://colab.research.google.com/github/kripet1/Datathon/blob/main/Copy_of_try3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
from datasets import load_dataset
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Load dataset
dataset = load_dataset('rotten_tomatoes')
train_texts, train_labels = dataset['train']['text'], dataset['train']['label']
test_texts, test_labels = dataset['test']['text'], dataset['test']['label']

# Preprocess texts
def preprocess_texts(texts):
    cleaned_texts = [text.lower() for text in texts]  # Add more preprocessing steps as needed
    return cleaned_texts

train_texts = preprocess_texts(train_texts)
test_texts = preprocess_texts(test_texts)

# Tokenize texts
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# Padding
max_length = max(len(x) for x in train_sequences)  # You might choose a fixed length based on distribution
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [4]:
!pip install datasets


In [3]:
from datasets import load_dataset
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
import numpy as np
import re

def clean_text(text):
    """
    Perform basic cleaning and preprocessing on text data.
    """
    text = text.lower()  # Lowercasing
    text = re.sub(r'\W', ' ', text)  # Remove all non-word characters
    text = re.sub(r'\s+', ' ', text)  # Replace all runs of whitespaces with a single space
    return text

# Load the Rotten Tomatoes movie review dataset
dataset = load_dataset('rotten_tomatoes')
train_texts, train_labels = dataset['train']['text'], dataset['train']['label']
test_texts, test_labels = dataset['test']['text'], dataset['test']['label']

# Preprocess the texts
train_texts = [clean_text(text) for text in train_texts]
test_texts = [clean_text(text) for text in test_texts]

# Tokenization and padding
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)
max_length = 120  # Choose a reasonable maximum length for padding

train_sequences = tokenizer.texts_to_sequences(train_texts)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')

test_sequences = tokenizer.texts_to_sequences(test_texts)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# Model definition
model = Sequential([
    Embedding(10000, 16, input_length=max_length),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(24, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Training
model.fit(train_padded, np.array(train_labels), epochs=10, validation_data=(test_padded, np.array(test_labels)))

# Evaluation
loss, accuracy = model.evaluate(test_padded, np.array(test_labels))
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

# Predictions
predictions = model.predict(test_padded)
binary_predictions = [1 if pred > 0.5 else 0 for pred in predictions.flatten()]

# Save predictions to CSV
results_df = pd.DataFrame({'index': range(len(binary_predictions)), 'pred': binary_predictions})
results_df.to_csv('static_we/results.csv', index=False)
print("Predictions saved to results.csv.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 conv1d (Conv1D)             (None, 116, 128)          10368     
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 24)                3096      
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 173489 (677.69 KB)
Trainable params: 173489 (677.69 KB)
Non-trainable params: 0 (0.00 Byte)
________________

KeyboardInterrupt: 

In [6]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, LSTM, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def clean_text(text):
    """
    Enhanced cleaning and preprocessing of text data.
    """
    text = text.lower()  # Lowercasing
    text = re.sub(r'<.*?>', '', text)  # Removing HTML tags
    text = re.sub(r'\d+', '', text)  # Removing numbers
    text = re.sub(r'\W', ' ', text)  # Remove all non-word characters
    text = re.sub(r'\s+', ' ', text)  # Replace all runs of whitespaces with single space
    return text.strip()

# Load the dataset
dataset = load_dataset('rotten_tomatoes')
train_texts, train_labels = dataset['train']['text'], dataset['train']['label']
test_texts, test_labels = dataset['test']['text'], dataset['test']['label']

# Preprocess the texts
train_texts = [clean_text(text) for text in train_texts]
test_texts = [clean_text(text) for text in test_texts]

# Tokenization and padding
tokenizer = Tokenizer(num_words=20000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)
max_length = 150  # Adjust based on your dataset's distribution

train_sequences = tokenizer.texts_to_sequences(train_texts)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')

test_sequences = tokenizer.texts_to_sequences(test_texts)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense, LSTM

model = Sequential([
    Embedding(20000, 32, input_length=max_length),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    LSTM(64, return_sequences=True),
    Dropout(0.5),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


# Implement early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Training
history = model.fit(train_padded, np.array(train_labels), epochs=20, validation_data=(test_padded, np.array(test_labels)), callbacks=[early_stopping])

# Evaluation
loss, accuracy = model.evaluate(test_padded, np.array(test_labels))
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

# Predictions
predictions = model.predict(test_padded)
binary_predictions = [1 if pred > 0.5 else 0 for pred in predictions.flatten()]

# Save predictions to CSV
results_df = pd.DataFrame({'index': range(len(binary_predictions)), 'pred': binary_predictions})
results_df.to_csv('static_we/results.csv', index=False)
print("Predictions saved to results.csv.")


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 150, 32)           640000    
                                                                 
 dropout_3 (Dropout)         (None, 150, 32)           0         
                                                                 
 conv1d_2 (Conv1D)           (None, 146, 64)           10304     
                                                                 
 lstm_1 (LSTM)               (None, 146, 64)           33024     
                                                                 
 dropout_4 (Dropout)         (None, 146, 64)           0         
                                                                 
 global_max_pooling1d_2 (Gl  (None, 64)                0         
 obalMaxPooling1D)                                               
                                                      

OSError: Cannot save file into a non-existent directory: 'static_we'